In [1]:
import os
import time
import random

import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
from albumentations.pytorch import ToTensorV2
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score


if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🔥 Using GPU: {gpu_name}")
    torch.backends.cudnn.benchmark = True  # 입력 크기 고정이면 더 빠름
else:
    device = torch.device("cpu")
    print("⚠️ GPU not available, using CPU.")

🔥 Using GPU: NVIDIA GeForce RTX 3090


In [2]:
import os, random, numpy as np, torch

# ------------------------
# Reproducibility & GPU Optimization
# ------------------------
SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# ⚙️ cuDNN settings
torch.backends.cudnn.deterministic = False  # True면 재현성↑, 속도↓
torch.backends.cudnn.benchmark = True       # 입력 크기 고정이면 속도↑
torch.backends.cudnn.enabled = True

# ⚡ Device check
if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🔥 Using GPU: {gpu_name}")
else:
    device = torch.device("cpu")
    print("⚠️ Using CPU. (No CUDA detected)")


🔥 Using GPU: NVIDIA GeForce RTX 3090


In [3]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import pandas as pd
import os

class ImageDataset(Dataset):
    def __init__(self, csv_or_df, path, transform=None):
        # csv 경로(str) or DataFrame 모두 허용
        if isinstance(csv_or_df, str):
            self.df = pd.read_csv(csv_or_df).values
        else:
            self.df = csv_or_df.values

        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df[idx]
        img = np.array(Image.open(os.path.join(self.path, name)).convert("RGB"))
        if self.transform:
            img = self.transform(image=img)["image"]
        return img, target



In [4]:
from tqdm import tqdm
import torch
from sklearn.metrics import accuracy_score, f1_score

def train_one_epoch(loader, model, optimizer, loss_fn, device, scaler=None):
    model.train()
    train_loss = 0.0
    preds_list, targets_list = [], []

    pbar = tqdm(loader, desc="Training", leave=False)

    for images, targets in pbar:
        images, targets = images.to(device, non_blocking=True), targets.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        # ⚡ Mixed Precision Training
        with torch.cuda.amp.autocast(enabled=(scaler is not None)):
            preds = model(images)
            loss = loss_fn(preds, targets)

        # ✅ backward (scaler 있을 때와 없을 때 모두 지원)
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        # Logging
        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())
        pbar.set_postfix(loss=f"{loss.item():.4f}")

    # Epoch metrics
    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }


In [5]:
# ------------------------
# ⚡ Device setting
# ------------------------
if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name(0)
    print(f"🔥 Using GPU: {gpu_name}")
    torch.backends.cudnn.benchmark = True  # 입력 크기 고정이면 속도 ↑
else:
    device = torch.device("cpu")
    print("⚠️ Using CPU. (No CUDA detected)")

# ------------------------
# ⚙️ Data config
# ------------------------
data_path = "../../../data/raw/"

# ------------------------
# 🧠 Model config
# ------------------------
# 문서 분류 정확도 우선 세팅 (ConvNeXt-Tiny)
model_name = "convnext_tiny.fb_in22k_ft_in1k"  # ImageNet22k → 1k fine-tuned 가중치 사용
IMG_CHANNELS = 3

# ------------------------
# 📈 Training config (RTX 3090 최적화)
# ------------------------
img_size = 384            # 문서 이미지면 384~512 추천
LR = 5e-4                 # ConvNeXt-Tiny에도 잘 맞는 값
EPOCHS = 15               # 약간 더 긴 학습 추천 (10 → 15)
BATCH_SIZE = 64           # ConvNeXt는 EfficientNet보다 메모리 조금 더 씀
num_workers = 8           # CPU 병렬 처리 강화
pin_memory = True
persistent_workers = True
prefetch_factor = 4       # DataLoader 성능 개선

# ------------------------
# 💡 Notes
# ------------------------
# - ConvNeXt-Tiny는 구조적으로 EfficientNet보다 깊고 넓은 모델이라,
#   GPU 사용량은 약 1.5~2배 높지만, 문서(텍스트/도장/레이아웃) 분류 성능이 훨씬 좋음.
# - AMP 사용 시 속도는 거의 동일, 메모리 절약 효과 큼.
# - 모델 로드는 아래처럼 하면 됨:
#     import timm
#     model = timm.create_model(model_name, pretrained=True, num_classes=NUM_CLASSES)
#     model.to(device)


🔥 Using GPU: NVIDIA GeForce RTX 3090


In [6]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# ------------------------
# 🧾 Train Transform (Document-Optimized)
# ------------------------
trn_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),

    # 살짝의 기울기 및 스케일 변화 (스캔 오차 흉내)
    A.ShiftScaleRotate(
        shift_limit=0.01,
        scale_limit=0.05,
        rotate_limit=2,
        p=0.3
    ),

    # 스캔 노이즈 및 블러 효과 (문서 품질 다양화)
    A.OneOf([
        A.MotionBlur(blur_limit=3, p=0.5),
        A.GaussNoise(var_limit=(5, 20), p=0.5),
    ], p=0.2),

    # 밝기·대비는 살짝만 조정 (폰카/스캐너 편차 대응)
    A.RandomBrightnessContrast(
        brightness_limit=0.05,
        contrast_limit=0.05,
        p=0.3
    ),

    # 대비 정규화 (문서 글자 선명하게)
    A.CLAHE(clip_limit=2.0, p=0.2),

    # 정규화 (ImageNet 기준 유지)
    A.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    ),

    ToTensorV2(),
])

# ------------------------
# 🧾 Validation / Test Transform (Matching Style)
# ------------------------
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.CLAHE(clip_limit=2.0, p=1.0),  # 밝기 대비 정규화 적용
    A.Normalize(
        mean=(0.5, 0.5, 0.5),
        std=(0.5, 0.5, 0.5)
    ),
    ToTensorV2(),
])


/data/ephemeral/home/py310/lib/python3.10/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
/tmp/ipykernel_85164/3393421690.py:21: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5, 20), p=0.5),


In [7]:
from sklearn.model_selection import train_test_split

# 1️⃣ train.csv 로드
full_df = pd.read_csv("../../../data/raw/train.csv")

# 2️⃣ train/val 분리 (stratify로 클래스 균형 유지)
trn_df, val_df = train_test_split(
    full_df, test_size=0.2, stratify=full_df['target'], random_state=42
)

# 3️⃣ Dataset 정의
trn_dataset = ImageDataset(trn_df, "../../../data/raw/train/", transform=trn_transform)
val_dataset = ImageDataset(val_df, "../../../data/raw/train/", transform=tst_transform)
tst_dataset = ImageDataset("../../../data/raw/sample_submission.csv",
                           "../../../data/raw/test/", transform=tst_transform)



In [8]:
from torch.utils.data import DataLoader

# Train loader
trn_loader = DataLoader(
    trn_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=pin_memory,
    persistent_workers=persistent_workers,
    prefetch_factor=prefetch_factor,
    drop_last=True
)

# Validation loader
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=pin_memory,
    persistent_workers=persistent_workers,
    prefetch_factor=prefetch_factor
)

# Test loader (inference 전용)
tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=pin_memory,
    persistent_workers=persistent_workers,
    prefetch_factor=prefetch_factor
)


In [9]:
import torch
import torch.nn as nn
import timm
from timm.optim import create_optimizer_v2

# ------------------------
# 🧠 Load Model
# ------------------------

model = timm.create_model(
    model_name,
    pretrained=True,
    num_classes=17,
    in_chans=3,
    drop_path_rate=0.1
).to(device)

# 기존 head 초기화 후 새로 정의
in_features = model.num_features  # ← get_classifier()보다 안전
model.head = nn.Sequential(
    nn.AdaptiveAvgPool2d(1),  # (B, C, H, W) → (B, C, 1, 1)
    nn.Flatten(1),            # (B, C, 1, 1) → (B, C)
    nn.LayerNorm(in_features),
    nn.Dropout(0.3),
    nn.Linear(in_features, 17)
).to(device)


# ------------------------
# 💥 Loss Function
# ------------------------
loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)

# ------------------------
# ⚙️ Optimizer (Lion)
# ------------------------
optimizer = create_optimizer_v2(
    model,
    opt='lion',
    lr=3e-4,            # 살짝 낮춰 안정적 학습
    weight_decay=1e-4,
)

# ------------------------
# 📈 Scheduler (OneCycleLR)
# ------------------------
from torch.optim.lr_scheduler import OneCycleLR

scheduler = OneCycleLR(
    optimizer,
    max_lr=3e-4,
    steps_per_epoch=len(trn_loader),
    epochs=EPOCHS,
    pct_start=0.1,
    anneal_strategy='cos',
)

# ------------------------
# ⚡ AMP Scaler
# ------------------------
scaler = torch.cuda.amp.GradScaler()


/tmp/ipykernel_85164/1904133478.py:61: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [10]:
from tqdm import tqdm
import torch
from sklearn.metrics import accuracy_score, f1_score

@torch.no_grad()
def validate(loader, model, loss_fn, device):
    model.eval()
    val_loss, preds_list, targets_list = 0.0, [], []

    pbar = tqdm(loader, desc="Validating", leave=False)
    for images, targets in pbar:
        images, targets = images.to(device, non_blocking=True), targets.to(device, non_blocking=True)
        with torch.amp.autocast("cuda"):
            preds = model(images)
            loss = loss_fn(preds, targets)

        val_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).cpu().numpy())
        targets_list.extend(targets.cpu().numpy())
        pbar.set_postfix(loss=f"{loss.item():.4f}")

    val_loss /= len(loader)
    val_acc = accuracy_score(targets_list, preds_list)
    val_f1 = f1_score(targets_list, preds_list, average="macro")

    return {"val_loss": val_loss, "val_acc": val_acc, "val_f1": val_f1}


def train_one_epoch(loader, model, optimizer, loss_fn, device, scaler=None, scheduler=None):
    model.train()
    train_loss = 0.0
    preds_list, targets_list = [], []

    pbar = tqdm(loader, desc="Training", leave=False)

    for images, targets in pbar:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True)

        optimizer.zero_grad(set_to_none=True)

        # ⚡ Mixed Precision
        with torch.cuda.amp.autocast(enabled=(scaler is not None)):
            preds = model(images)
            loss = loss_fn(preds, targets)

        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            loss.backward()
            optimizer.step()

        # ⚙️ OneCycleLR은 iteration 단위로 호출해야 함
        if scheduler is not None:
            scheduler.step()

        train_loss += loss.item()
        preds_list.extend(preds.argmax(dim=1).detach().cpu().numpy())
        targets_list.extend(targets.detach().cpu().numpy())

        pbar.set_postfix(loss=f"{loss.item():.4f}")

    train_loss /= len(loader)
    train_acc = accuracy_score(targets_list, preds_list)
    train_f1 = f1_score(targets_list, preds_list, average='macro')

    return {
        "train_loss": train_loss,
        "train_acc": train_acc,
        "train_f1": train_f1,
    }

best_f1 = 0.0


In [11]:

for epoch in range(EPOCHS):
    # 1️⃣ Train (이제 scheduler는 내부에서 step됨)
    train_metrics = train_one_epoch(trn_loader, model, optimizer, loss_fn, device, scaler, scheduler)

    # 2️⃣ Validation
    val_metrics = validate(val_loader, model, loss_fn, device)

    # 3️⃣ Logging
    print(f"\n[Epoch {epoch+1}/{EPOCHS}]")
    print(f"Train | Loss: {train_metrics['train_loss']:.4f} | "
          f"Acc: {train_metrics['train_acc']:.4f} | "
          f"F1: {train_metrics['train_f1']:.4f}")
    print(f"Valid | Loss: {val_metrics['val_loss']:.4f} | "
          f"Acc: {val_metrics['val_acc']:.4f} | "
          f"F1: {val_metrics['val_f1']:.4f}")

    # 4️⃣ Save Best Model
    if val_metrics["val_f1"] > best_f1:
        best_f1 = val_metrics["val_f1"]
        torch.save({
            "epoch": epoch + 1,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
            "val_loss": val_metrics["val_loss"],
            "val_f1": val_metrics["val_f1"],
        }, f"./best_model_f1_{best_f1:.4f}_epoch{epoch+1}.pt")
        print(f"✅ Best model updated! (F1: {best_f1:.4f})")




Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_85164/4174820261.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
/data/ephemeral/home/py310/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:182: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(
                                                                                                                         


[Epoch 1/15]
Train | Loss: 2.7537 | Acc: 0.1686 | F1: 0.1529
Valid | Loss: 1.8748 | Acc: 0.5318 | F1: 0.4280
✅ Best model updated! (F1: 0.4280)


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_85164/4174820261.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               


[Epoch 2/15]
Train | Loss: 1.8670 | Acc: 0.5016 | F1: 0.4556
Valid | Loss: 1.2616 | Acc: 0.7197 | F1: 0.6483
✅ Best model updated! (F1: 0.6483)


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_85164/4174820261.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               

...


[Epoch 13/15]
Train | Loss: 0.6391 | Acc: 0.9836 | F1: 0.9830
Valid | Loss: 0.6804 | Acc: 0.9490 | F1: 0.9469


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_85164/4174820261.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               


[Epoch 14/15]
Train | Loss: 0.6373 | Acc: 0.9844 | F1: 0.9844
Valid | Loss: 0.6745 | Acc: 0.9554 | F1: 0.9545


Training:   0%|                                                                                                                         | 0/19 [00:00<?, ?it/s]/tmp/ipykernel_85164/4174820261.py:43: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(scaler is not None)):
                                                                                                                                                               


[Epoch 15/15]
Train | Loss: 0.6385 | Acc: 0.9844 | F1: 0.9841
Valid | Loss: 0.6738 | Acc: 0.9554 | F1: 0.9545


# 6. Inference & Save File
* 테스트 이미지에 대한 추론을 진행하고, 결과 파일을 저장합니다.

In [12]:
from tqdm import tqdm
import pandas as pd
import torch

# ------------------------
# 💾 모델 저장
# ------------------------
torch.save(model.state_dict(), "./final_model_convnext_tiny.pt")
print("✅ Model weights saved as final_model_convnext_tiny.pt")

# ------------------------
# 🧠 Inference
# ------------------------
preds_list = []

model.eval()
with torch.no_grad():
    for images, _ in tqdm(tst_loader, desc="🔍 Inference"):
        images = images.to(device, non_blocking=True)
        with torch.cuda.amp.autocast():
            preds = model(images)
        preds_list.extend(preds.argmax(dim=1).cpu().numpy())

# ------------------------
# 📊 결과 DataFrame 생성
# ------------------------
pred_df = pd.DataFrame(tst_dataset.df, columns=["ID", "target"])
pred_df["target"] = preds_list

# ------------------------
# 🔎 샘플 제출 파일과 순서 검증
# ------------------------
sample_submission_df = pd.read_csv("../../../data/raw/sample_submission.csv")
assert (sample_submission_df["ID"] == pred_df["ID"]).all(), "⚠️ ID 순서 불일치!"

# ------------------------
# 💾 최종 CSV 저장
# ------------------------
save_path = "pred_6.csv"
pred_df.to_csv(save_path, index=False)
print(f"🎯 Inference complete! Saved to {save_path}")


✅ Model weights saved as final_model_convnext_tiny.pt


🔍 Inference:   0%|                                                                                                                     | 0/50 [00:00<?, ?it/s]/tmp/ipykernel_85164/265959225.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
🔍 Inference: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:06<00:00,  8.11it/s]

🎯 Inference complete! Saved to pred_6.csv
